In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2019-04-08 02:18:08--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  65.0MB/s    in 4.7s    

2019-04-08 02:18:13 (49.9 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [0]:
!mv tiny-imagenet-200.zip tinyimagenet.zip

In [0]:
!unzip -qq 'tinyimagenet.zip'

In [0]:
!mv tiny-imagenet-200 tinyimagenet

In [0]:
import keras
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Activation, add
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu
from tensorflow.keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Lambda
from tensorflow.keras.regularizers import l2
from keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau

from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit
import os

# input image dimensions
img_rows, img_cols = 64, 64
# The tinyimagenet images are RGB.
img_channels = 3

batch_size = 2048
num_classes = 200
num_epoch = 50
data_augmentation = True

path='./tinyimagenet'

Using TensorFlow backend.


In [0]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-7)
csv_logger = CSVLogger('/content/gdrive/My Drive/TPUCustomImageAugFinalv2.csv')
filepath="/content/gdrive/My Drive/epochs_TPUCustomImageAugFinalv2:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [0]:
num_outputs = 200
input = Input(shape=(64, 64, 3,))

conv1 = Conv2D(64, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_1')(input)										#32,32,64			3
bn1 = BatchNormalization(name='norm_1')(conv1)
act1 = Activation('relu')(bn1)
conv2 = Conv2D(64, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_2')(act1) 										#32,32,64			5

#layer1
bn2 = BatchNormalization(name='norm_2')(conv2)
act2 = Activation('relu')(bn2)
conv3 = Conv2D(64, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_3')(act1) 										#32,32,64			7								
#add
add1 = add([conv2, conv3]) 																	                                                                                #32,32,64

bn3 = BatchNormalization(name='norm_3')(add1)
act3 = Activation('relu')(bn3)
conv4 = Conv2D(128, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_4')(act3)								#32,32,128		9								
bn4 = BatchNormalization(name='norm_4')(conv4)  
act4 = Activation('relu')(bn4)
pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")(act4)											                                          #16,16,128			18

#add
shortconv1 = Conv2D(128, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_1')(add1)				#32,32,128			20
shortconv11 = Conv2D(128, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_11')(shortconv1) #16,16,128			20
add2 = add([shortconv11, pool1])																						                                                                  #16,16,128

#layer2
bn5 = BatchNormalization(name='norm_5')(add2)
act5 = Activation('relu')(bn5)
conv5 = Conv2D(256, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_5')(act5)								#16,16,256	    24 *+4
bn6 = BatchNormalization(name='norm_6')(conv5)
act6 = Activation('relu')(bn6)

shortconv2 = Conv2D(256, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_2')(add2)				#16,16,256		  28
#add
add3 = add([shortconv2, act6])																						                                                                  #16,16,256

bn7 = BatchNormalization(name='norm_7')(add3)
act7 = Activation('relu')(bn7)
conv6 = SeparableConv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform', bias_initializer='zeros', depthwise_regularizer=l2(1.e-4), pointwise_regularizer=None, bias_regularizer=None, activity_regularizer=None, depthwise_constraint=None, pointwise_constraint=None, bias_constraint=None)(act7)
bn8 = BatchNormalization(name='norm_8')(conv6)																		                                                          #16,16,512			32
act8 = Activation('relu')(bn8)
conv7 = Conv2D(512, (3,3), strides=(2,2), padding='same', kernel_regularizer=l2(1.e-4), name='conv_7')(act8)										#8,8,512			  36
#add
shortconv3 = Conv2D(512, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_3')(add3)	      #8,8,512			  44 *+8
shortconv33 = Conv2D(512, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_33')(shortconv3) #8,8,512			44
add4 = add([shortconv33, conv7])																						                                                                  #8,8,512


bn9 = BatchNormalization(name='norm_9')(add4)
act9 = Activation('relu')(bn9)
conv8 = SeparableConv2D(1024, (3, 3), strides=(1, 1), padding='same', activation='relu', depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform', bias_initializer='zeros', depthwise_regularizer=l2(1.e-4), pointwise_regularizer=None, bias_regularizer=None, activity_regularizer=None, depthwise_constraint=None, pointwise_constraint=None, bias_constraint=None)(act9)
bn10 = BatchNormalization(name='norm_10')(conv8)																		                                                        #8,8,1024			  60 *+16
act10 = Activation('relu')(bn10)
conv9 = Conv2D(1024, (3,3), strides=(2,2), padding='same', kernel_regularizer=l2(1.e-4), name='conv_9')(act10)									#4,4,1024			  76
#add
shortconv4 = Conv2D(1024, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_4')(add4)			#4,4,1024			  100 *+24
shortconv44 = Conv2D(1024, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_44')(shortconv4) #4,4,1024			100
add5 = add([shortconv44, conv9])																						                                                                  #4,4,1024

bn11 = BatchNormalization(name='norm_11')(add5)																		                                                          #4,4,1024			  164  
act11 = Activation('relu')(bn11)


conv15 = Conv2D(num_outputs, (1,1), strides=(1,1), name='conv_15', use_bias=False)(act11)
pool2 = GlobalAveragePooling2D()(conv15)
output = Activation('softmax')(pool2)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model = Model(inputs=[input], outputs=[output])

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.61.220.130:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8367284064408530391)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10854564726786272880)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5444045647809446164)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:

In [0]:
train_loss = tf.keras.losses.categorical_crossentropy

tpu_model.compile(
    optimizer='adam',
    loss=train_loss,
    metrics=['accuracy']
)

In [0]:
tpu_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalizationV1)   (None, 64, 64, 64)   256         conv_1[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 64)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Co

In [0]:
seq = iaa.Sequential(
      [
		  iaa.Sometimes(0.3, iaa.CoarseDropout((0.3), size_percent=(0.01))), #removing pixels in form of rectangles
      iaa.Sometimes(0.3, iaa.Crop(percent=(0, 0.3))), #cropping by 30%
		  iaa.Sometimes(0.3, iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})), #scale images to 80-120%, translate by -20 to 20% per axis
      iaa.Sometimes(0.3, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))), #sharpening the images using lightness value, high lightness value makes the image brighter
      iaa.Sometimes(0.3, iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0, 0.7)), #running edge detection kernel
                    iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0)),
                    ])),
      iaa.Sometimes(0.3, iaa.AdditiveGaussianNoise(scale=(0.1))), #adding noise to images
      iaa.Sometimes(0.1, iaa.GaussianBlur(sigma=(0, 0.5))) #making the images blurred
	  ],

      # do all of the above augmentations in random order
      random_order=True
  )

In [0]:
def lr_schedule(epoch):
    if epoch > 100:
        lr = 0.5e-5
    elif epoch > 75:
        lr = 1e-5
    elif epoch > 50:
        lr = 0.5e-4
    elif epoch > 30:
        lr = 1e-4
    else:
        lr = 1e-3
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [0]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    preprocessing_function = seq.augment_image
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tinyimagenet/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical')

Found 100000 images belonging to 200 classes.


In [0]:
val_data = pd.read_csv('./tinyimagenet/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tinyimagenet/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size)

Found 10000 images belonging to 200 classes.


In [0]:
tpu_model.fit_generator(train_generator, 
                        epochs=125, 
                        steps_per_epoch=int(100000//batch_size),
                        validation_steps=int(10000//batch_size), 
                        validation_data=validation_generator,
                        callbacks=[checkpoint, lr_reducer, csv_logger, lr_scheduler] )

Learning rate (from LearningRateScheduler):  0.001
Epoch 1/125
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(212,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(212, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(212, 200), dtype=tf.float32, name='activation_11_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.Adam object at 0x7f4556fd6668> []
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 26.906020879745483 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:CPU -> TPU 

In [0]:
seq = iaa.Sequential(
      [
      iaa.Sometimes(0.5, iaa.CoarseDropout((0.5), size_percent=(0.01))),
		  iaa.Sometimes(0.5, iaa.Crop(percent=(0, 0.6))),
		  iaa.Sometimes(0.4, iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, rotate=(-30, 30))),	  
      iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))),
      iaa.Sometimes(0.1, iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0, 0.7)),
                    iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0)),
                    ])),
      iaa.Sometimes(0.4, iaa.AdditiveGaussianNoise(scale=(0.1))),
      iaa.Sometimes(0.1, iaa.GaussianBlur(sigma=(0, 0.5)))
	  ],

      # do all of the above augmentations in random order
      random_order=True
  )

In [0]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    preprocessing_function = seq.augment_image
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
def lr_schedule(epoch):
    #Learning Rate Schedule
    #
    #Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    #Called automatically every epoch as part of callbacks during training.

    # Arguments
    #    epoch (int): The number of epochs

    # Returns
    #    lr (float32): learning rate
    #
    
    if epoch > 15:
        lr = 0.5e-5
    elif epoch > 5:
        lr = 1e-5
    else:
        lr = 0.5e-4
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [0]:
tpu_model.fit_generator(train_generator, 
                        epochs=20, 
                        steps_per_epoch=int(100000//batch_size),
                        validation_steps=int(10000//batch_size), 
                        validation_data=validation_generator,
                        callbacks=[checkpoint, lr_reducer, csv_logger, lr_scheduler] )

Learning rate (from LearningRateScheduler):  5e-05
Epoch 1/20
5/5 [==============================] - 14s 3s/step - loss: 2.0973 - acc: 0.5632

Epoch 00001: val_acc did not improve from 0.57350
49/49 [==============================] - 200s 4s/step - loss: 1.3079 - acc: 0.7484 - val_loss: 2.0973 - val_acc: 0.5632
Learning rate (from LearningRateScheduler):  5e-05
Epoch 2/20
5/5 [==============================] - 14s 3s/step - loss: 2.0778 - acc: 0.5696

Epoch 00002: val_acc did not improve from 0.57350
49/49 [==============================] - 194s 4s/step - loss: 1.3057 - acc: 0.7476 - val_loss: 2.0778 - val_acc: 0.5696
Learning rate (from LearningRateScheduler):  5e-05
Epoch 3/20
5/5 [==============================] - 14s 3s/step - loss: 2.0822 - acc: 0.5642

Epoch 00003: val_acc did not improve from 0.57350
49/49 [==============================] - 192s 4s/step - loss: 1.3009 - acc: 0.7484 - val_loss: 2.0822 - val_acc: 0.5642
Learning rate (from LearningRateScheduler):  5e-05
Epoch 4/20

In [0]:
tpu_model.save_weights('/content/gdrive/My Drive/savedWeightsFinal.hdf5')

INFO:tensorflow:Copying TPU weights to the CPU
INFO:tensorflow:TPU -> CPU lr: 4.999999873689376e-06
INFO:tensorflow:TPU -> CPU beta_1: 0.8999999761581421
INFO:tensorflow:TPU -> CPU beta_2: 0.9990000128746033
INFO:tensorflow:TPU -> CPU decay: 0.0
INFO:tensorflow:TPU -> CPU epsilon: 1e-07
INFO:tensorflow:TPU -> CPU amsgrad: False


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tinyimagenet/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=False)

Found 10000 images belonging to 200 classes.


In [0]:
steps=np.math.ceil(validation_generator.samples/validation_generator.batch_size)

In [0]:
Y_pred = tpu_model.predict_generator(validation_generator, steps)
y_pred = np.argmax(Y_pred, axis=1)

INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(256, 64, 64, 3), dtype=tf.float32, name='input_1_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'lr': 4.999999873689376e-06, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 18.859996557235718 secs
INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(226, 64, 64, 3), dtype=tf.float32, name='input_1_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 12.929059028625488 secs


In [0]:
from sklearn.metrics import classification_report
print(classification_report(validation_generator.classes, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86        50
           1       0.79      0.76      0.78        50
           2       0.62      0.56      0.59        50
           3       0.42      0.40      0.41        50
           4       0.51      0.56      0.53        50
           5       0.47      0.40      0.43        50
           6       0.67      0.68      0.67        50
           7       0.59      0.48      0.53        50
           8       0.81      0.78      0.80        50
           9       0.57      0.54      0.56        50
          10       0.47      0.52      0.50        50
          11       0.64      0.60      0.62        50
          12       0.72      0.72      0.72        50
          13       0.69      0.76      0.72        50
          14       0.69      0.72      0.71        50
          15       0.59      0.48      0.53        50
          16       0.42      0.38      0.40        50
          17       0.67    

In [0]:
import pandas as pd 
pd.DataFrame(y_pred).to_csv("predFinal.csv")

In [0]:
my_dict = validation_generator.class_indices

import csv


with open('indicesFinal.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, my_dict.keys())
    w.writeheader()
    w.writerow(my_dict)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tinyimagenet/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=False)

Found 100000 images belonging to 200 classes.


In [0]:
Y1_pred = tpu_model.predict_generator(train_generator, np.math.ceil(train_generator.samples/train_generator.batch_size))
y1_pred = np.argmax(Y1_pred, axis=1)

In [0]:
print(classification_report(train_generator.classes, y1_pred))

              precision    recall  f1-score   support

           0       0.43      0.57      0.49       500
           1       0.85      0.46      0.59       500
           2       0.37      0.41      0.39       500
           3       0.34      0.22      0.26       500
           4       0.53      0.22      0.32       500
           5       0.63      0.16      0.26       500
           6       0.52      0.28      0.36       500
           7       0.46      0.21      0.29       500
           8       0.59      0.35      0.44       500
           9       0.60      0.18      0.27       500
          10       0.36      0.26      0.30       500
          11       0.44      0.30      0.36       500
          12       0.52      0.41      0.46       500
          13       0.64      0.48      0.55       500
          14       0.58      0.38      0.46       500
          15       0.38      0.29      0.33       500
          16       0.35      0.33      0.34       500
          17       0.28    

In [0]:
train_generator.class_indices

{'n01443537': 0,
 'n01629819': 1,
 'n01641577': 2,
 'n01644900': 3,
 'n01698640': 4,
 'n01742172': 5,
 'n01768244': 6,
 'n01770393': 7,
 'n01774384': 8,
 'n01774750': 9,
 'n01784675': 10,
 'n01855672': 11,
 'n01882714': 12,
 'n01910747': 13,
 'n01917289': 14,
 'n01944390': 15,
 'n01945685': 16,
 'n01950731': 17,
 'n01983481': 18,
 'n01984695': 19,
 'n02002724': 20,
 'n02056570': 21,
 'n02058221': 22,
 'n02074367': 23,
 'n02085620': 24,
 'n02094433': 25,
 'n02099601': 26,
 'n02099712': 27,
 'n02106662': 28,
 'n02113799': 29,
 'n02123045': 30,
 'n02123394': 31,
 'n02124075': 32,
 'n02125311': 33,
 'n02129165': 34,
 'n02132136': 35,
 'n02165456': 36,
 'n02190166': 37,
 'n02206856': 38,
 'n02226429': 39,
 'n02231487': 40,
 'n02233338': 41,
 'n02236044': 42,
 'n02268443': 43,
 'n02279972': 44,
 'n02281406': 45,
 'n02321529': 46,
 'n02364673': 47,
 'n02395406': 48,
 'n02403003': 49,
 'n02410509': 50,
 'n02415577': 51,
 'n02423022': 52,
 'n02437312': 53,
 'n02480495': 54,
 'n02481823': 55,
 '

In [0]:
results = pd.DataFrame(validation_generator.classes, y_pred, columns=['Predicted']).reset_index()
results['True'] = results['index'] == results['Predicted']
results['True']=results['True'].astype(int)
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
results['predict_class'] = [label_map[k] for k in results['index']]
results = results.groupby('predict_class')['True'].sum().to_dict()

In [0]:
def get_class_weights(dict_labels_vs_samples, balanced=True):
  # param dict_labels_vs_samples: dict of 'label' vs. 'number of correct predictions'
  # param balanced: if True, will produce class weights considering max samples count - % of augmented images per class will be proportional to max count of samples 
  # param balanced: if False, will produce class weights considering average samples count - % of aumented images per class will be proportional to avg samples of classes
  
  keys = dict_labels_vs_samples.keys()
  values = list(dict_labels_vs_samples.values())
  total_samples = sum(values)
  num_classes = len(values)
  max_of_all_classes = max(values)
  average_of_all_classes = total_samples / num_classes
  multiplying_factor = 1
  
  if balanced:
    multiplying_factor = max_of_all_classes / average_of_all_classes
  
  print('total_samples: ', total_samples)
  print('num_classes: ', num_classes)
  print('max_of_all_classes: ', max_of_all_classes)
  print('multiplying_factor: ', multiplying_factor)
  
  class_weight = dict()

  for key in keys:
        score_for_class = (total_samples / (num_classes * dict_labels_vs_samples.get(key))) * multiplying_factor
        class_weight[key] = score_for_class
        
  return class_weight

In [0]:
weights = get_class_weights(results, True)
print(weights)

total_samples:  5714
num_classes:  200
max_of_all_classes:  45
multiplying_factor:  1.575078753937697
{'n01443537': 0.9999999999999999, 'n01629819': 1.1842105263157894, 'n01641577': 1.6071428571428574, 'n01644900': 2.25, 'n01698640': 1.6071428571428574, 'n01742172': 2.25, 'n01768244': 1.323529411764706, 'n01770393': 1.875, 'n01774384': 1.1538461538461537, 'n01774750': 1.6666666666666665, 'n01784675': 1.7307692307692308, 'n01855672': 1.5, 'n01882714': 1.25, 'n01910747': 1.1842105263157894, 'n01917289': 1.25, 'n01944390': 1.875, 'n01945685': 2.3684210526315788, 'n01950731': 1.25, 'n01983481': 1.6071428571428574, 'n01984695': 1.3636363636363635, 'n02002724': 1.323529411764706, 'n02056570': 1.3636363636363635, 'n02058221': 1.3636363636363635, 'n02074367': 0.9999999999999999, 'n02085620': 1.875, 'n02094433': 1.2162162162162162, 'n02099601': 1.5517241379310347, 'n02099712': 2.8125, 'n02106662': 1.8, 'n02113799': 2.647058823529412, 'n02123045': 1.7307692307692308, 'n02123394': 1.40625, 'n0212

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tinyimagenet/train', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', seed=42, classes=weights)

Found 100000 images belonging to 200 classes.


In [0]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tinyimagenet/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size)

Found 10000 images belonging to 200 classes.


In [0]:
def lr_schedule(epoch):
    if epoch > 100:
        lr = 0.5e-5
    elif epoch > 75:
        lr = 1e-5
    elif epoch > 50:
        lr = 0.5e-4
    elif epoch > 30:
        lr = 1e-4
    else:
        lr = 1e-3
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [0]:
tpu_model.fit_generator(train_generator, 
                        epochs=125, 
                        steps_per_epoch=int(100000//batch_size),
                        validation_steps=int(10000//batch_size), 
                        validation_data=validation_generator,
                        callbacks=[checkpoint, lr_scheduler, csv_logger, lr_reducer] )

Learning rate (from LearningRateScheduler):  0.001
Epoch 1/125
5/5 [==============================] - 15s 3s/step - loss: 3.8297 - acc: 0.3621

Epoch 00001: val_acc did not improve from 0.57350
49/49 [==============================] - 215s 4s/step - loss: 3.6923 - acc: 0.2731 - val_loss: 3.8297 - val_acc: 0.3621
Learning rate (from LearningRateScheduler):  0.001
Epoch 2/125
5/5 [==============================] - 13s 3s/step - loss: 5.0913 - acc: 0.2538

Epoch 00002: val_acc did not improve from 0.57350
49/49 [==============================] - 208s 4s/step - loss: 3.5333 - acc: 0.3023 - val_loss: 5.0913 - val_acc: 0.2538
Learning rate (from LearningRateScheduler):  0.001
Epoch 3/125
5/5 [==============================] - 13s 3s/step - loss: 3.1596 - acc: 0.4047

Epoch 00003: val_acc did not improve from 0.57350
49/49 [==============================] - 209s 4s/step - loss: 3.5447 - acc: 0.3021 - val_loss: 3.1596 - val_acc: 0.4047
Learning rate (from LearningRateScheduler):  0.001
Epoch 4

In [0]:
tpu_model.load_weights('/content/gdrive/My Drive/epochs_TPUCustomImageAugFinal:058-val_acc:0.590.hdf5')

In [0]:
weights = {'n01443537': 0.9999999999999999, 'n01629819': 1.1842105263157894, 'n01641577': 1.6071428571428574, 'n01644900': 2.25, 'n01698640': 1.6071428571428574, 'n01742172': 2.25, 'n01768244': 1.323529411764706, 'n01770393': 1.875, 'n01774384': 1.1538461538461537, 'n01774750': 1.6666666666666665, 'n01784675': 1.7307692307692308, 'n01855672': 1.5, 'n01882714': 1.25, 'n01910747': 1.1842105263157894, 'n01917289': 1.25, 'n01944390': 1.875, 'n01945685': 2.3684210526315788, 'n01950731': 1.25, 'n01983481': 1.6071428571428574, 'n01984695': 1.3636363636363635, 'n02002724': 1.323529411764706, 'n02056570': 1.3636363636363635, 'n02058221': 1.3636363636363635, 'n02074367': 0.9999999999999999, 'n02085620': 1.875, 'n02094433': 1.2162162162162162, 'n02099601': 1.5517241379310347, 'n02099712': 2.8125, 'n02106662': 1.8, 'n02113799': 2.647058823529412, 'n02123045': 1.7307692307692308, 'n02123394': 1.40625, 'n02124075': 2.3684210526315788, 'n02125311': 1.8, 'n02129165': 1.2162162162162162, 'n02132136': 1.323529411764706, 'n02165456': 1.125, 'n02190166': 1.5, 'n02206856': 1.3636363636363635, 'n02226429': 1.5517241379310347, 'n02231487': 2.25, 'n02233338': 1.956521739130435, 'n02236044': 2.142857142857143, 'n02268443': 1.8, 'n02279972': 1.0465116279069766, 'n02281406': 1.1538461538461537, 'n02321529': 1.4516129032258065, 'n02364673': 1.4516129032258065, 'n02395406': 2.8125, 'n02403003': 1.8, 'n02410509': 1.1842105263157894, 'n02415577': 1.5517241379310347, 'n02423022': 1.1842105263157894, 'n02437312': 1.1538461538461537, 'n02480495': 1.4516129032258065, 'n02481823': 1.323529411764706, 'n02486410': 1.8, 'n02504458': 1.2857142857142858, 'n02509815': 1.1538461538461537, 'n02666196': 1.956521739130435, 'n02669723': 1.25, 'n02699494': 1.5517241379310347, 'n02730930': 2.25, 'n02769748': 1.5517241379310347, 'n02788148': 3.214285714285715, 'n02791270': 2.0454545454545454, 'n02793495': 1.5, 'n02795169': 2.25, 'n02802426': 1.5517241379310347, 'n02808440': 1.8, 'n02814533': 1.6666666666666665, 'n02814860': 1.125, 'n02815834': 2.0454545454545454, 'n02823428': 1.6666666666666665, 'n02837789': 1.323529411764706, 'n02841315': 1.6666666666666665, 'n02843684': 1.7307692307692308, 'n02883205': 2.647058823529412, 'n02892201': 1.25, 'n02906734': 2.3684210526315788, 'n02909870': 3.4615384615384617, 'n02917067': 0.9999999999999999, 'n02927161': 1.4516129032258065, 'n02948072': 1.875, 'n02950826': 2.25, 'n02963159': 2.0454545454545454, 'n02977058': 1.6071428571428574, 'n02988304': 2.647058823529412, 'n02999410': 2.5, 'n03014705': 1.8, 'n03026506': 1.2857142857142858, 'n03042490': 1.40625, 'n03085013': 1.40625, 'n03089624': 1.3636363636363635, 'n03100240': 1.6666666666666665, 'n03126707': 1.6666666666666665, 'n03160309': 1.7307692307692308, 'n03179701': 1.5517241379310347, 'n03201208': 1.40625, 'n03250847': 2.647058823529412, 'n03255030': 3.75, 'n03355925': 1.40625, 'n03388043': 1.4516129032258065, 'n03393912': 1.0714285714285714, 'n03400231': 2.3684210526315788, 'n03404251': 2.5, 'n03424325': 2.0454545454545454, 'n03444034': 1.1538461538461537, 'n03447447': 1.0465116279069766, 'n03544143': 1.5, 'n03584254': 1.6071428571428574, 'n03599486': 1.323529411764706, 'n03617480': 2.3684210526315788, 'n03637318': 1.6666666666666665, 'n03649909': 2.142857142857143, 'n03662601': 1.0714285714285714, 'n03670208': 1.5517241379310347, 'n03706229': 1.5, 'n03733131': 1.1538461538461537, 'n03763968': 2.142857142857143, 'n03770439': 2.0454545454545454, 'n03796401': 1.4516129032258065, 'n03804744': 2.647058823529412, 'n03814639': 2.0454545454545454, 'n03837869': 1.097560975609756, 'n03838899': 2.647058823529412, 'n03854065': 1.40625, 'n03891332': 1.5517241379310347, 'n03902125': 1.5517241379310347, 'n03930313': 1.2162162162162162, 'n03937543': 2.142857142857143, 'n03970156': 6.42857142857143, 'n03976657': 3.75, 'n03977966': 1.1538461538461537, 'n03980874': 1.875, 'n03983396': 3.4615384615384617, 'n03992509': 1.5, 'n04008634': 2.8125, 'n04023962': 2.647058823529412, 'n04067472': 2.647058823529412, 'n04070727': 1.4516129032258065, 'n04074963': 1.956521739130435, 'n04099969': 1.875, 'n04118538': 1.0714285714285714, 'n04133789': 1.6666666666666665, 'n04146614': 0.9999999999999999, 'n04149813': 1.2857142857142858, 'n04179913': 1.7307692307692308, 'n04251144': 1.875, 'n04254777': 1.2857142857142858, 'n04259630': 1.6666666666666665, 'n04265275': 1.6071428571428574, 'n04275548': 1.323529411764706, 'n04285008': 1.5, 'n04311004': 1.8, 'n04328186': 1.5, 'n04356056': 1.6666666666666665, 'n04366367': 1.6071428571428574, 'n04371430': 2.3684210526315788, 'n04376876': 3.4615384615384617, 'n04398044': 2.0454545454545454, 'n04399382': 1.3636363636363635, 'n04417672': 1.40625, 'n04456115': 1.4516129032258065, 'n04465501': 1.5517241379310347, 'n04486054': 1.125, 'n04487081': 1.0714285714285714, 'n04501370': 1.6666666666666665, 'n04507155': 3.4615384615384617, 'n04532106': 1.4516129032258065, 'n04532670': 1.097560975609756, 'n04540053': 1.1538461538461537, 'n04560804': 2.5, 'n04562935': 1.125, 'n04596742': 1.5517241379310347, 'n04597913': 4.090909090909091, 'n06596364': 1.1538461538461537, 'n07579787': 1.8, 'n07583066': 1.323529411764706, 'n07614500': 2.25, 'n07615774': 2.25, 'n07695742': 1.6666666666666665, 'n07711569': 1.7307692307692308, 'n07715103': 1.6071428571428574, 'n07720875': 1.2162162162162162, 'n07734744': 1.25, 'n07747607': 1.40625, 'n07749582': 1.3636363636363635, 'n07753592': 1.4516129032258065, 'n07768694': 1.125, 'n07871810': 1.6666666666666665, 'n07873807': 1.2162162162162162, 'n07875152': 1.5, 'n07920052': 1.0714285714285714, 'n09193705': 1.323529411764706, 'n09246464': 1.5, 'n09256479': 1.2162162162162162, 'n09332890': 2.25, 'n09428293': 1.6071428571428574, 'n12267677': 1.8}

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tinyimagenet/train', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', seed=42, classes=weights)

Found 100000 images belonging to 200 classes.


In [0]:
def lr_schedule(epoch):    
    if epoch > 36:
        lr = 0.5e-5
    elif epoch > 11:
        lr = 1e-5
    else:
        lr = 0.5e-4
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [0]:
tpu_model.fit_generator(train_generator, 
                        epochs=61, 
                        steps_per_epoch=int(100000//batch_size),
                        validation_steps=int(10000//batch_size), 
                        validation_data=validation_generator,
                        callbacks=[checkpoint, lr_scheduler, csv_logger, lr_reducer] )

Learning rate (from LearningRateScheduler):  5e-05
Epoch 1/61
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(256,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(256, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(256, 200), dtype=tf.float32, name='activation_11_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'lr': 4.999999873689376e-05, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.Adam object at 0x7f9548915b38> []
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 25.99946880340576 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:CPU -> TPU lr

In [0]:
def lr_schedule(epoch):
    #Learning Rate Schedule
    #
    #Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    #Called automatically every epoch as part of callbacks during training.

    # Arguments
    #    epoch (int): The number of epochs

    # Returns
    #    lr (float32): learning rate
    #
    
    if epoch > 100:
        lr = 0.5e-5
    elif epoch > 75:
        lr = 1e-5
    elif epoch > 50:
        lr = 0.5e-4
    elif epoch > 30:
        lr = 1e-4
    else:
        lr = 1e-3
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [0]:
tpu_model.fit_generator(train_generator, 
                        epochs=105, 
                        steps_per_epoch=int(100000//batch_size),
                        validation_steps=int(10000//batch_size), 
                        validation_data=validation_generator,
                        callbacks=[checkpoint, lr_scheduler, csv_logger, lr_reducer] )

Learning rate (from LearningRateScheduler):  0.001
Epoch 1/105
5/5 [==============================] - 13s 3s/step - loss: 3.2875 - acc: 0.4325

Epoch 00001: val_acc did not improve from 0.59350
49/49 [==============================] - 203s 4s/step - loss: 3.1485 - acc: 0.3657 - val_loss: 3.2875 - val_acc: 0.4325
Learning rate (from LearningRateScheduler):  0.001
Epoch 2/105
5/5 [==============================] - 13s 3s/step - loss: 3.4450 - acc: 0.4116

Epoch 00002: val_acc did not improve from 0.59350
49/49 [==============================] - 200s 4s/step - loss: 3.2488 - acc: 0.3544 - val_loss: 3.4450 - val_acc: 0.4116
Learning rate (from LearningRateScheduler):  0.001
Epoch 3/105
5/5 [==============================] - 13s 3s/step - loss: 3.3236 - acc: 0.4346

Epoch 00003: val_acc did not improve from 0.59350
49/49 [==============================] - 200s 4s/step - loss: 3.2450 - acc: 0.3609 - val_loss: 3.3236 - val_acc: 0.4346
Learning rate (from LearningRateScheduler):  0.001
Epoch 4